In [1]:
import nltk
from nltk.corpus import brown, gutenberg, conll2000
from nltk.corpus import wordnet as wn
train_data = conll2000.tagged_sents('train.txt') + brown.tagged_sents(categories=['news'])
test_data = conll2000.tagged_sents('test.txt')

patterns = [
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # simple past
    (r'.*es$', 'VBZ'),                 # 3rd singular present
    (r'.*ould$', 'MD'),                # modals
    (r'.*\'s$', 'NN$'),                # possessive nouns
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'^-?[0-9]+(\.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN')                      # nouns (default)
]

regexp_tagger = nltk.RegexpTagger(patterns)
unigram_tagger = nltk.UnigramTagger(train_data, backoff=regexp_tagger)
bigram_tagger = nltk.BigramTagger(train_data, backoff=unigram_tagger)


In [2]:
# Chunker
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == len(sentence)-1:
        nextword, nextpos = "<END>", "<END>"
    else:
        nextword, nextpos = sentence[i+1]
    return {"pos": pos,
            "word": word,
            "prevpos": prevpos,
            "nextpos": nextpos, 
            "prevpos+pos": "%s+%s" % (prevpos, pos),  
            "pos+nextpos": "%s+%s" % (pos, nextpos),
            "tags-since-dt": tags_since_dt(sentence, i)}  

def tags_since_dt(sentence, i):
    tags = set()
    for word, pos in sentence[:i]:
        if pos == 'DT':
            tags = set()
        else:
            tags.add(pos)
    return '+'.join(sorted(tags))



class ConsecutiveNPChunkTagger(nltk.TaggerI): 

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) 
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train( 
            train_set, algorithm=None, trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI): 
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [3]:
test_sents = conll2000.chunked_sents('test.txt')
train_sents = conll2000.chunked_sents('train.txt')
chunker = ConsecutiveNPChunker(train_sents)


/home/emil/anaconda3/lib/python3.7/site-packages/nltk/classify/maxent.py:1386: RuntimeWarning: overflow encountered in power
  exp_nf_delta = 2 ** nf_delta
/home/emil/anaconda3/lib/python3.7/site-packages/nltk/classify/maxent.py:1388: RuntimeWarning: invalid value encountered in multiply
  sum1 = numpy.sum(exp_nf_delta * A, axis=0)
/home/emil/anaconda3/lib/python3.7/site-packages/nltk/classify/maxent.py:1389: RuntimeWarning: invalid value encountered in multiply
  sum2 = numpy.sum(nf_exp_nf_delta * A, axis=0)


In [235]:
from urllib.request import urlopen
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.wsd import lesk
import json
import re
import inflect

engine = inflect.engine()

def simplify(sent, bigrams):
    sents = []
    start = 0
    for i in range(len(bigrams)):
        pair = bigrams[i]
        w1, t1 = pair[0]
        w2, t2 = pair[1]
        if w1 == ',' and t2.startswith('W'):
            sents.append(sent[start:i])
            start = i + 1
            end = start
            while end < len(sent) and sent[end] != ',':
                end += 1
            sents.append(sent[start:end])
            if end < len(sent):
                sents[-2].extend(sent[end + 1:])
            sents[-1][0] = 'He/she/it/they/there'
            break
    if not sents:
        sents.append(sent)
    return sents
    
def refine(sent):
    if sent[-1] == ',':
        sent[-1] = '.'
    if not sent[0].istitle():
        sent[0] = sent[0].title()
    if sent[-1].isalpha():
        sent.append('.')
    return sent

def decompose(sent, tagger):
    sents = []
    tagged_sent = tagger.tag(sent)
    bigrams = list(nltk.trigrams(tagged_sent))
    start = 0
    for i in range(start, len(bigrams)):
        triple = bigrams[i]
        w1, t1 = triple[0]
        w2, t2 = triple[1]
        w3, t3 = triple[2]
        cond = ((t2 == 'RB' or t2 == 'PRP' or t2.startswith('NN')) and (t3.startswith('VB') or t3 == 'BEDZ'))
        if t1 == 'CC' and cond:
            sents.append(sent[start:i])
            sent = sent[i + 1:]
            start = i + 1
        elif i == len(bigrams) - 1:
            sents.append(sent)
    output = []
    for sent in sents:
        output.append(refine(sent))
        
    simplified_sents = []
    length_so_far = 0
    for sent in sents:
        for simple_sent in simplify(sent, bigrams[length_so_far : length_so_far + len(sent)]):
            simplified_sents.append(simple_sent)
        length_so_far += len(sent)
    
    simplified_sents = [refine(sent) for sent in simplified_sents]
    return simplified_sents

conversion_map = defaultdict(list)
conversion_map['adjective'] = ['a', 's']
conversion_map['adverb'] = ['r']
conversion_map['noun'] = ['n']
conversion_map['verb'] = ['v']
conversion_map['plural noun'] = ['n']
conversion_map['transitive verb'] = ['v']
stopwords = stopwords.words('english')

# storing the most frequent n words, in the decreasing order of frequency
def all_words(filename):
    f = open(filename)
    output = []
    i = 0
    for line in f:
        if i == 0:
            i += 1
            continue
        word, freq = line.split(',')
        output.append(word)
        i += 1
    f.close()
    return output
        
all_words = all_words('unigram_freq.csv')

def freq_index(word, all_words):
    for i in range(len(all_words)):
        if all_words[i] == word:
            return i
    return -1

# word sense disambiguation
def wsd(sent, word):
    return lesk(sent, word)

def define(word):
    url = 'https://api.dictionaryapi.dev/api/v2/entries/en_US/' + word
    try:
        data = urlopen(url).read()
        dictionary = json.loads(data)
        return dictionary[0]
    except:
        return None

def is_list_of_dicts(d):
    return isinstance(d, list) and all(isinstance(item, dict) for item in d)

def get_synonyms(word):
    definition = define(word)
    if not definition:
        return None
    meanings = definition['meanings']
    output = defaultdict(list)
    for meaning in meanings:
        pos = meaning['partOfSpeech']
        for definition in meaning['definitions']:
            if 'synonyms' in definition:
                output[pos].append([definition['definition'], definition['synonyms']])
    return output

def same_lemma(word1, word2):
    try:
        def1, def2 = define(word1), define(word2)
        return def1['word'] == def2['word']
    except:
        return False

def similarity_score(word_def, syn_def, syn_list):
    for syn in syn_list:
        index = freq_index(syn, all_words)
        if (syn in word_def) and (index > -1) and (index < 10000):
            return syn
    sent1 = nltk.word_tokenize(word_def)
    sent2 = nltk.word_tokenize(syn_def)
    tagged_sent1 = nltk.pos_tag(sent1)
    tagged_sent2 = nltk.pos_tag(sent2)
    word_def = [(word.lower(), tag) for (word, tag) in tagged_sent1 if word.isalpha() and not word in stopwords]
    syn_def = [(word.lower(), tag) for (word, tag) in tagged_sent2 if word.isalpha() and not word in stopwords]
    score = 0
    for (word1, tag1) in word_def:
        for (word2, tag2) in syn_def:
            if same_lemma(word1, word2):
                score += 1
                if tag1.startswith('V') and tag2.startswith('V'):
                    score += 0.25
    return score


file = open('irregular_verbs.txt')
irreg_verbs = {}
for line in file:
    forms = line.split()
    verb = forms[0]
    vbd = forms[1]
    vbn = forms[2]
    vbg = (forms[0] + 'ing') if not (forms[0][-1] in 'aeiou') else (forms[0][:-1] + 'ing')
    vbz = forms[0] + 's' #(forms[0] + 'es') if not (forms[0][-1] in 'aeiou') else (forms[0] + 's')
    irreg_verbs[verb] = {'VBD': vbd, 'VBZ': vbz, 'VBG': vbg, 'VBN': vbn, 'VBP': verb}
    
def pluralize(noun):
    return engine.plural(noun)


def comparative(adj):
    if len(adj) > 5:
        return adj
    return (adj + 'r') if adj[-1] == 'e' else (adj + 'er')
    
    
def superlative(adj):
    if len(adj) > 5:
        return adj
    return (adj + 'st') if adj[-1] == 'e' else (adj + 'est')
    
            
def handle_word(original_word, original_tag, letter, sub_word):
    cond1 = original_tag.startswith('JJ') and letter == 's'
    cond2 = original_tag.lower().startswith(letter)
    if cond2:
        if letter == 'v':
            for end in ['ed', 'ing']:
                if original_word.endswith(end):
                    return ((sub_word + end) if sub_word[-1] != 'e' else (sub_word[:-1] + end))
            else:
                return irreg_verbs[sub_word][original_tag]
        elif letter == 'n':
            return sub_word
    elif cond1:
        if original_word.endswith('er'):
            return comparative(sub_word)
        elif original_word.endswith('est'):
            return superlative(sub_word)
        else:
            return sub_word
    return sub_word

def easiest_synonym(tagged_sent, i, all_words):
    word = tagged_sent[i][0]
    basic_form = word
    d = define(word)
    if d:
        basic_form = d['word']
    fi = freq_index(basic_form, all_words)
    if fi < 10000:
        return word
    elif word.isalpha() and i > 0 and i < len(tagged_sent) and word.istitle():
        return word
    
    synset = wsd([w for (w, t) in tagged_sent], word)
    if synset:
        word_tag = synset.pos()
        word_definition = synset.definition()
        synonyms = get_synonyms(word) # returns a map, where keys are pos-tags
        if synonyms:
            easiest = None
            index = 333333
            for pos_tag in synonyms:
                if word_tag in conversion_map[pos_tag] and (tagged_sent[i][1].lower().startswith(word_tag) or 
                                                            (tagged_sent[i][1].startswith('JJ') and word_tag == 's')):
                    max_score = 0
                    syn_list = []
                    for n in range(len(synonyms[pos_tag])):
                        definition, synonym_list = synonyms[pos_tag][n]
                        score = similarity_score(word_definition, definition, synonym_list)
                        if isinstance(score, str):
                            return score
                        elif score > max_score:
                            max_score = score
                            syn_list = synonym_list
                    if max_score == 0 and len(synonyms[pos_tag]) == 1:
                        for j in range(len(synonyms[pos_tag][0][1])):
                            current_index = freq_index(synonyms[pos_tag][0][1][j], all_words)
                            if current_index > -1 and current_index < 10000:
                                easiest = synonyms[pos_tag][0][1][j]  
                    else:   
                        for synonym in syn_list:
                            current = freq_index(synonym, all_words)
                            if current > -1 and current < 10000:
                                easiest = synonym
                                break
                    easiest = handle_word(word, tagged_sent[i][1], word_tag, easiest)
            if easiest:
                return easiest
    return word

def simplify_text(text, tagger, all_words):
    sents = nltk.sent_tokenize(text)
    tokenized_sents = [nltk.word_tokenize(sent) for sent in sents]
    modified_sents = []
    for sent in tokenized_sents:
        a = decompose(sent, tagger)
        for simple_sent in a:
            modified_sents.append(simple_sent)
    output = [[easiest_synonym(tagger.tag(sent), i, all_words) for i in range(len(sent))] 
            for sent in modified_sents]
    return output

def simplify_and_convert_to_text(text, tagger, all_words):
    sents = simplify_text(text, tagger, all_words)
    text = ''
    for sent in sents:
        for i in range(len(sent)):
            if sent[i] == "'s":
                text = text[:-1]
            text += sent[i]
            if (i < len(sent) - 1 and not (sent[i + 1] in '.,?!:;')) or i == len(sent) - 1:
                text += ' '
    return text

class NLTK_Tagger(nltk.TaggerI): 
    def tag(self, sentence):
        return nltk.pos_tag(sentence)
    
nltk_tagger = NLTK_Tagger()

In [228]:
text = open('Animals_wiki.txt').read()
print(text)

Animals (also called Metazoa) are multicellular eukaryotic organisms that form the biological kingdom Animalia. With few exceptions, animals consume organic material, breathe oxygen, are able to move, can reproduce sexually, and grow from a hollow sphere of cells, the blastula, during embryonic development. Over 1.5 million living animal species have been described—of which around 1 million are insects—but it has been estimated there are over 7 million animal species in total. Animals range in length from 8.5 micrometres (0.00033 in) to 33.6 metres (110 ft). They have complex interactions with each other and their environments, forming intricate food webs. The scientific study of animals is known as zoology.





In [216]:
print(simplify_and_convert_to_text(text, nltk_tagger, all_words))

Animals ( also called Metazoa ) are multicellular eukaryotic bodies that form the biological kingdom Animalia. With few exceptions, animals eat organic material, breathe oxygen, are able to move, can reproduce sexually, and grow from a hollow sphere of cells, the blastula, during incomplete development. Over 1.5 million living animal species have been described—of which around 1 million are insects—but it has been estimated there are over 7 million animal species in total. Animals range in length from 8.5 micrometres ( 0.00033 in ) to 33.6 metres ( 110 ft ). They have complex interactions with each other and their environments, forming complex food webs. The scientific study of animals is known as zoology. 


In [236]:
text = open('Isaac_Newton_2.txt').read()
print(text)

In June 1661, he was admitted to Trinity College, Cambridge, on the recommendation of his uncle Rev William Ayscough, who had studied there. He started as a subsizar—paying his way by performing valet's duties—until he was awarded a scholarship in 1664, guaranteeing him four more years until he could get his MA. At that time, the college's teachings were based on those of Aristotle, whom Newton supplemented with modern philosophers such as Descartes, and astronomers such as Galileo and Thomas Street, through whom he learned of Kepler's work. He set down in his notebook a series of "Quaestiones" about mechanical philosophy as he found it. In 1665, he discovered the generalised binomial theorem and began to develop a mathematical theory that later became calculus. Soon after Newton had obtained his BA degree in August 1665, the university temporarily closed as a precaution against the Great Plague. Although he had been undistinguished as a Cambridge student, Newton's private studies at h

In [237]:
print(simplify_and_convert_to_text(text, nltk_tagger, all_words))

In June 1661, he was admitted to Trinity College, Cambridge, on the recommendation of his uncle Rev William Ayscough. He/She/It/They/There had studied there. He started as a subsizar—paying his way by performing man's duties—until he was awarded a scholarship in 1664, guaranteeing him four more years until he could get his MA. At that time, the college's teachings were based on those of Aristotle and astronomers such as Galileo and Thomas Street, through whom he learned of Kepler's work. He/She/It/They/There Newton supplemented with modern philosophers such as Descartes. He set down in his notebook a series of `` Quaestiones '' about mechanical philosophy as he found it. In 1665, he discovered the generalised binomial theorem and began to develop a mathematical theory that later became calculus. Soon after Newton had obtained his BA degree in August 1665, the university temporarily closed as a insurance against the Great Plague. Although he had been undistinguished as a Cambridge stude

In [241]:
text = open('Roman_Empire.txt').read()
print(text)

The predecessor state of the Roman Empire, the Roman Republic (which had replaced Rome's monarchy in the 6th century BC) became severely destabilized in a series of civil wars and political conflicts. In the mid-1st century BC, Julius Caesar was appointed as perpetual dictator and then assassinated in 44 BC. Civil wars and proscriptions continued, culminating in the victory of Octavian, Caesar's adopted son, over Mark Antony and Cleopatra at the Battle of Actium in 31 BC. The following year Octavian conquered Ptolemaic Egypt, ending the Hellenistic period that had begun with the conquests of Alexander the Great of Macedon in the 4th century BC. Octavian's power then became unassailable, and in 27 BC the Roman Senate formally granted him overarching power and the new title Augustus, effectively making him the first Roman emperor.



In [242]:
print(simplify_and_convert_to_text(text, nltk_tagger, all_words))

The predecessor state of the Roman Empire, the Roman Republic ( which had replaced Rome's monarchy in the 6th century BC ) became seriously upseted in a series of civil wars and political conflicts. In the mid-1st century BC, Julius Caesar was appointed as endless dictator. Then assassinated in 44 BC. Civil wars. ban continued, peaking in the victory of Octavian, Caesar's adopted son, over Mark Antony and Cleopatra at the Battle of Actium in 31 BC. The following year Octavian conquered Ptolemaic Egypt, ending the Hellenistic period that had begun with the conquests of Alexander the Great of Macedon in the 4th century BC. Octavian's power then became unassailable, and in 27 BC the Roman Senate formally granted him overarching power and the new title Augustus, effectively making him the first Roman emperor. 
